## Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, ParameterGrid, validation_curve, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from factor_analyzer import FactorAnalyzer
import pingouin as pg
%matplotlib inline

## Main Functions

In [2]:
# Function for checking missing value 
def missing_val(df):
    d = df.isna().sum().to_dict()
    missing_k = [] 
    missing_v = []
    missing_p = []
    for k, v in d.items():
        if v > 0:
            missing_k.append(k)
            missing_v.append(v)
        else:
            continue
    for v in d.values():
        if v > 0:
            per = round((v/len(df) * 100), 4)
            missing_p.append(per)
        else:
            continue
    missing_df = pd.DataFrame(data=zip(missing_k, missing_v, missing_p), columns=['Column', 'Total Nulll', 'Missing %'])
    return missing_df

def save_csv(df, fname):
    df.to_csv('/Users/Ming/Documents/GitHub/wellcome_dissertation/WellcomePy/csv_data/Wellcome/' + fname, index=False)

def open_csv(fname):
    df = pd.read_csv('/Users/Ming/Documents/GitHub/wellcome_dissertation/WellcomePy/csv_data/Wellcome/' + fname)
    return df

# Create factor analysis with loadings
def factor_analysis(df, factor_num):
    fa = FactorAnalyzer(factor_num, rotation='varimax')
    fa.fit(df)
    loads = fa.loadings_
    ev, v = fa.get_eigenvalues()
    return (loads, ev, v)

# Create top 5 variables in each factor
def factor_dataframe (df, loads):
    factors = ['Factor ' + str(i) for i in range(1, (loads.shape[1] + 1))]
    fadf = pd.DataFrame(data=loads, columns=factors)
    fadf.insert(loc=0, column='VAR', value=df.columns)
    for i in range (1, (loads.shape[1] + 1)):
        factor_top5 = fadf.sort_values(by='Factor ' + str(i), ascending=False)[['VAR','Factor ' + str(i)]].iloc[:5]
        factor_top5.set_index('VAR', inplace=True) 
        print('\n')
        print(factor_top5)

# Create scree plot
def factor_plot(df, ev, fname):
    fig = plt.figure(figsize=(5,5))
    plt.scatter(range(1,df.shape[1]+1),ev)
    plt.plot(range(1,df.shape[1]+1),ev)
    plt.title('Scree Plot')
    plt.xlabel('Factors')
    plt.ylabel('Eigenvalue')
    plt.grid()
    fig.savefig('/Users/Ming/Documents/GitHub/wellcome_dissertation/WellcomePy/py_image/' + fname)

def idx_dev(df):
    df = df.loc[:, ['VAC_IMPTCH', 'VAC_SF', 'VAC_EFF']]
    df = df[(df['VAC_IMPTCH'] != 99) & (df['VAC_SF'] != 99) & (df['VAC_EFF'] != 99)]
    # Factor Analysis
    # Insert prepared dataframe (must not contain single unique value/Null in each column), number of factors, and loads
    loads, ev, v = factor_analysis(df, 1)
    factor_dataframe(df, loads)

    # Cronbach Alpha test
    factor = df[['VAC_IMPTCH', 'VAC_SF', 'VAC_EFF']]
    factor_alpha = pg.cronbach_alpha(factor)
    print(factor_alpha)

# Check abnormal data for each country
def check_ctry(col):
    ndf = pd.DataFrame()
    ndf['Country'] = df[df[col].isna()]['CTRY'].unique()
    ndf = ndf.replace(country)
    ndf['Country Code'] = df[df[col].isna()]['CTRY'].unique()
    ndf =ndf.sort_values(by=['Country']).reset_index(drop=True)
    return ndf

# Check if a country have all missing value for certain variable. Abnormal means the country has all missing values for that variable, normal means the country only have a portion of missing values for that variable.
def check_ctry_unique(ndf, var):
    df = open_csv('full_data.csv')
    normal_ctry = []
    notnormal_ctry = []
    for ctry in ndf['Country Code']:
        missing = df[df['CTRY'] == ctry][var].unique()
        if len(missing) <= 1:
            notnormal_ctry.append(ctry)
        else:
            normal_ctry.append(ctry)
    mapnormal = [country.get(item, item) for item in normal_ctry]
    mapabnormal = [country.get(item, item) for item in notnormal_ctry] 
    return (normal_ctry, mapnormal, notnormal_ctry, mapabnormal)

# Assign age category
def age_category(df):
    for idx, row in df[df['AGE_CAT'].isna()].iterrows():
        if 15 <= df.loc[idx, 'AGE'] <= 29:
            df.loc[idx, 'AGE_CAT'] = 1
        elif 30 <= df.loc[idx, 'AGE'] <= 49:
            df.loc[idx, 'AGE_CAT'] = 2
        else:
            df.loc[idx, 'AGE_CAT'] = 3
    return df

# Check train and test percentage
def calculate_percentage(df1, df2):
    return len(df1)/ (len(df2) + len(df1)) *100

## Import Data and Basic Exploration

In [245]:
# df = pd.read_excel('/Users/Ming/Documents/GitHub/wellcome_dissertation/WellcomePy/excel_data/full_data.xlsx')
# df.to_csv('/Users/Ming/Documents/GitHub/wellcome_dissertation/WellcomePy/csv_data/full_data.csv', index=False)

# Replace all empty cell with NaN
df = df.replace(' ', np.NaN)

# # Change Data Type
df['TRUSCI_IDX'] = df['TRUSCI_IDX'].astype('float64')
df.info()
print('\n', '-- Dtypes Count --', '\n', df.dtypes.value_counts())

In [246]:
# Import Data
df = open_csv('full_data.csv')
df.info()
print('\n', '-- Dtypes Count --', '\n', df.dtypes.value_counts())

## Preprocessing

### Initial Preprocessing

In [65]:
# Import dataset 
df = open_csv('full_data.csv')

In [108]:
# Drop NAT_WGT, POP_WGT, FLD_DATE, YEAR
df = df.drop(['NAT_WGT', 'POP_WGT', 'FLD_DATE', 'YEAR'], axis=1)

In [67]:
# Impute SC_SS and SC_UNI with 97 since if no attended antecedent school means no next level schooling
for idx, row in df[df['SC_PS'] == 97][['SC_SS', 'SC_UNI']].iterrows():
    df.loc[idx, ['SC_SS', 'SC_UNI']] = 97

for idx, row in df[df['SC_SS'] == 97][['SC_UNI']].iterrows():
    df.loc[idx, ['SC_UNI']] = 97

In [71]:
# Impute TRUSCI_INX by replacing NA with 0
for idx, row in df[df['TRUSCI_IDX'].isna()].iterrows():
    df.loc[idx, ['TRUSCI_IDX']] = 0 

In [72]:
# Drop VAC_IMPTCH, VAC_SF, and VAC_EFF with NA since we are interested on the attitude of those who know vaccine only
df = df.dropna(subset=['VAC_IMPTCH'], how='all')

In [75]:
# Drop ANY_CH with NA
df = df.dropna(subset=['ANY_CH'], how='all')

In [77]:
# Drop TRU_NATGOV, TRU_GOVADV, RELIG, SC_DISRELIG, BLV_SCRELIG, HHI
df = df.drop(['TRU_NATGOV', 'TRU_GOVADV', 'RELIG', 'SC_DISRELIG', 'BLV_SCRELIG', 'HHI'], axis=1)

In [80]:
# Drop Northern Cyprus
df = df.drop(df.loc[df['CTRY']==202].index)

In [112]:
# Split into with child (with CH_RVAC) and no child (without CH_RVAC)
df_ch = df.dropna(subset=['CH_RVAC'], how='all')
df_noch = df[df['CH_RVAC'].isna()].drop(['ANY_CH','CH_RVAC'], axis=1)

In [121]:
# Split into train and test set before preprocessing
train_ch = df_ch.sample(frac=0.8, random_state=32)
test_ch = df_ch.drop(train_ch.index)
train_noch = df_noch.sample(frac=0.8, random_state=32)
test_noch = df_noch.drop(train_noch.index)

In [140]:
# Save initial pre-processed data
save_csv(df, 'fpp_df.csv')
save_csv(df_ch, 'df_ch.csv')
save_csv(df_noch, 'df_noch.csv')
save_csv(train_ch.reset_index(drop=True), 'train_ch.csv')
save_csv(test_ch.reset_index(drop=True), 'test_ch.csv')
save_csv(train_noch.reset_index(drop=True), 'train_noch.csv')
save_csv(test_noch.reset_index(drop=True), 'test_noch.csv')

### Second Preprocessing

In [153]:
# After imputed AGE, EDU, LIV_AR, HHI, and EMP_STAT
imptrain_ch = open_csv('imptrain_ch.csv')
imptest_ch = open_csv('imptest_ch.csv')
imptrain_noch = open_csv('imptrain_noch.csv')
imptest_noch = open_csv('imptest_noch.csv')

In [156]:
# Assign age into category to impute AGE_CAT
imptrain_ch = age_category(imptrain_ch)
imptest_ch = age_category(imptest_ch)
imptrain_noch = age_category(imptrain_noch)
imptest_noch = age_category(imptest_noch)

In [165]:
# Change data type for AGE_CAT
imptrain_ch['AGE_CAT'] = imptrain_ch['AGE_CAT'].astype('int64')
imptest_ch['AGE_CAT'] = imptest_ch['AGE_CAT'].astype('int64')
imptrain_noch['AGE_CAT'] = imptrain_noch['AGE_CAT'].astype('int64')
imptest_noch['AGE_CAT'] = imptest_noch['AGE_CAT'].astype('int64')

In [171]:
# Final train and test set for child and no child 
save_csv(imptrain_ch, 'ftrain_ch.csv')
save_csv(imptest_ch, 'ftest_ch.csv')
save_csv(imptrain_noch, 'ftrain_noch.csv')
save_csv(imptest_noch, 'ftest_noch.csv')

## Data Description

In [150]:
df = open_csv('full_data.csv')
info_dict = {'Column':df.columns, 'Count':df.count(), 'Type':df.dtypes}
dfinfo = pd.DataFrame(info_dict).reset_index(drop=True)
save_csv(dfinfo, 'dfinfo.csv')

In [397]:
# General data description plot
df = open_csv('full_data.csv')
info_dict = {'Column':df.columns, 'Count':df.count(), 'Type':df.dtypes}
dfinfo = pd.DataFrame(info_dict).reset_index(drop=True)
dfdes = sns.catplot(x='Count', y='Column', hue='Type', data=dfinfo, kind='bar')
dfdes.set(xlabel='Number of Records', ylabel='Variable')
dfdes.fig.set_size_inches(10,10)
dfdes.fig.suptitle('WGM 2018 Data Overview', y=1.0, fontsize=14)
dfdes.savefig('/Users/Ming/Documents/GitHub/wellcome_dissertation/WellcomePy/py_image/dfdes.png')
plt.close()

In [4]:
country = {1:'United States', 2:'Egypt', 3:'Morocco', 4:'Lebanon', 5:'Saudi Arabia', 6:'Jordan', 8:'Turkey', 9:'Pakistan', 10:'Indonesia', 11:'Bangladesh', 12:'United Kingdom', 13:'France', 14:'Germany', 15:'Netherlands', 16:'Belgium', 17:'Spain', 18:'Italy', 19:'Poland', 20:'Hungary', 21:'Czech Republic', 22:'Romania', 23:'Sweden', 24:'Greece', 25:'Denmark', 26:'Iran', 28:'Singapore', 29:'Japan', 30:'China', 31:'India', 32:'Venezuela', 33:'Brazil', 34:'Mexico', 35:'Nigeria', 36:'Kenya', 37:'Tanzania', 38:'Israel', 39:'Palestinian Territories', 40:'Ghana', 41:'Uganda', 42:'Benin', 43:'Madagascar', 44:'Malawi', 45:'South Africa', 46:'Canada', 47:'Australia', 48:'Philippines', 49:'Sri Lanka', 50:'Vietnam', 51:'Thailand', 52:'Cambodia', 53:'Laos', 54:'Myanmar', 55:'New Zealand', 57:'Botswana', 60:'Ethiopia', 61:'Mali', 62:'Mauritania', 63:'Mozambique', 64:'Niger', 65:'Rwanda', 66:'Senegal', 67:'Zambia', 68:'South Korea', 69:'Taiwan', 70:'Afghanistan', 71:'Belarus', 72:'Georgia', 73:'Kazakhstan', 74:'Kyrgyzstan', 75:'Moldova', 76:'Russia', 77:'Ukraine', 78:'Burkina Faso', 79:'Cameroon', 80:'Sierra Leone', 81:'Zimbabwe', 82:'Costa Rica', 83:'Albania', 84:'Algeria', 87:'Argentina', 88:'Armenia', 89:'Austria', 90:'Azerbaijan', 96:'Bolivia', 97:'Bosnia and Herzegovina', 99:'Bulgaria', 100:'Burundi', 103:'Chad', 104:'Chile', 105:'Colombia', 106:'Comoros', 108:'Republic of Congo', 109:'Croatia', 111:'Cyprus', 114:'Dominican Republic', 115:'Ecuador', 116:'El Salvador', 119:'Estonia', 121:'Finland', 122:'Gabon', 124:'Guatemala', 125:'Guinea', 128:'Haiti', 129:'Honduras', 130:'Iceland',131:'Iraq', 132:'Ireland', 134:'Ivory Coast', 137:'Kuwait', 138:'Latvia', 140:'Liberia', 141:'Libya', 143:'Lithuania', 144:'Luxembourg', 145:'Macedonia', 146:'Malaysia', 148:'Malta', 150:'Mauritius', 153:'Mongolia', 154:'Montenegro', 155:'Namibia', 157:'Nepal', 158:'Nicaragua', 160:'Norway', 163:'Panama', 164:'Paraguay', 165:'Peru', 166:'Portugal', 173:'Serbia', 175:'Slovakia', 176:'Slovenia', 183:'Eswatini', 184:'Switzerland', 185:'Tajikistan', 186:'The Gambia', 187:'Togo', 190:'Tunisia', 191:'Turkmenistan', 193:'United Arab Emirates', 194:'Uruguay', 195:'Uzbekistan', 197:'Yemen', 198:'Kosovo', 202:'Northern Cyprus'}

In [418]:
# Check number of records across every country
df = open_csv('full_data.csv')
df.insert(loc=1, column='CTRY_NAME', value=df['CTRY'].replace(country))
ctrycount = pd.DataFrame(data=zip(df['CTRY'].value_counts().index, df['CTRY_NAME'].value_counts().index, df['CTRY_NAME'].value_counts().values), columns=['Country Code', 'Country', 'Count'])

In [394]:
fig, ax = plt.subplots(figsize=(10,10))
ctryplot = sns.scatterplot(data=ctrycount, x="Country Code", y="Count", ax=ax, color='#EEB03F')
for line in range(0,ctrycount.shape[0]):
    if (ctrycount.loc[line, 'Country'] == 'Iceland') and ctrycount.loc[line, 'Count'] == 500:
        ctryplot.text(ctrycount['Country Code'][line]-11.5, ctrycount['Count'][line], ctrycount['Country'][line], horizontalalignment='left', size='medium', color='black', weight='regular')
    elif (ctrycount.loc[line, 'Country'] == 'Haiti') and ctrycount.loc[line, 'Count'] == 500:
        ctryplot.text(ctrycount['Country Code'][line]+1.5, ctrycount['Count'][line], ctrycount['Country'][line], horizontalalignment='left', size='medium', color='black', weight='regular')
    elif ctrycount.loc[line, 'Count'] > 1000:
        ctryplot.text(ctrycount['Country Code'][line]+0.5, ctrycount['Count'][line], ctrycount['Country'][line], horizontalalignment='left', size='medium', color='black', weight='regular')
    else:
        continue
ctryplot.set(xlabel='Country', ylabel='Number of Sample Data')
fig = ctryplot.get_figure()
fig.suptitle('Number of sample data across countries', y=0.94, fontsize=14)
fig.savefig('/Users/Ming/Documents/GitHub/wellcome_dissertation/WellcomePy/py_image/ctrycount.png')
plt.close()

In [131]:
# Data Distribution WGM 2018 (long)
fig = plt.figure(figsize=(20,30))
gs = GridSpec(11,5, wspace=0.2, hspace=0.5, figure=fig)
count = 0
while count <= 55:
    if count < 6:
        for c, d in zip(range(5), range(5,10)):
            ax = fig.add_subplot(gs[0,c])
            sns.violinplot(x=df.iloc[:,[d]], ax=ax)
            ax.set_title(df.iloc[:,[d]].columns.values[0])
            count += 1
    elif 5 < count < 11:
        for c, d in zip(range(5), range(10,15)):
            ax = fig.add_subplot(gs[1,c])
            sns.violinplot(x=df.iloc[:,[d]], ax=ax)
            ax.set_title(df.iloc[:,[d]].columns.values[0])
            count += 1
    elif 10 < count < 16:
        for c, d in zip(range(5), range(15,20)):
            ax = fig.add_subplot(gs[2,c])
            sns.violinplot(x=df.iloc[:,[d]], ax=ax)
            ax.set_title(df.iloc[:,[d]].columns.values[0])
            count += 1
    elif 15 < count < 21:
        for c, d in zip(range(5), range(20,25)):
            ax = fig.add_subplot(gs[3,c])
            sns.violinplot(x=df.iloc[:,[d]], ax=ax)
            ax.set_title(df.iloc[:,[d]].columns.values[0])
            count += 1
    elif 20 < count < 26:
        for c, d in zip(range(5), range(25,30)):
            ax = fig.add_subplot(gs[4,c])
            sns.violinplot(x=df.iloc[:,[d]], ax=ax)
            ax.set_title(df.iloc[:,[d]].columns.values[0])
            count += 1
    elif 25 < count < 31:
        for c, d in zip(range(5), range(30,35)):
            ax = fig.add_subplot(gs[5,c])
            sns.violinplot(x=df.iloc[:,[d]], ax=ax)
            ax.set_title(df.iloc[:,[d]].columns.values[0])
            count += 1
    elif 30 < count < 36:
        for c, d in zip(range(5), range(35,40)):
            ax = fig.add_subplot(gs[6,c])
            sns.violinplot(x=df.iloc[:,[d]], ax=ax)
            ax.set_title(df.iloc[:,[d]].columns.values[0])
            count += 1

    elif 35 < count < 41:
        for c, d in zip(range(5), range(40,45)):
            ax = fig.add_subplot(gs[7,c])
            sns.violinplot(x=df.iloc[:,[d]], ax=ax)
            ax.set_title(df.iloc[:,[d]].columns.values[0])
            count += 1
    
    elif 40 < count < 46:
        for c, d in zip(range(5), range(45,50)):
            ax = fig.add_subplot(gs[8,c])
            sns.violinplot(x=df.iloc[:,[d]], ax=ax)
            ax.set_title(df.iloc[:,[d]].columns.values[0])
            count += 1

    elif 45 < count < 51:
        for c, d in zip(range(5), range(50,55)):
            ax = fig.add_subplot(gs[9,c])
            sns.violinplot(x=df.iloc[:,[d]], ax=ax)
            ax.set_title(df.iloc[:,[d]].columns.values[0])
            count += 1

    else:
        for c, d in zip(range(5), range(55,60)):
            ax = fig.add_subplot(gs[10,c])
            sns.violinplot(x=df.iloc[:,[d]], ax=ax)
            ax.set_title(df.iloc[:,[d]].columns.values[0])
            count += 1

fig.suptitle("WGM 2018 Data Distribution", fontsize=16, y=0.9)
fig.tight_layout()
fig.savefig('/Users/Ming/Documents/GitHub/wellcome_dissertation/WellcomePy/py_image/datadist.png')
plt.close()

In [147]:
# Data Distribution WGM 2018 (wide)
fig = plt.figure(figsize=(30,20))
gs = GridSpec(5,11, wspace=0.2, hspace=0.5, figure=fig)
count = 0
while count <= 55:
    if count < 12:
        for c, d in zip(range(12), range(5,16)):
            ax = fig.add_subplot(gs[0,c])
            sns.violinplot(x=df.iloc[:,[d]], ax=ax)
            ax.set_title(df.iloc[:,[d]].columns.values[0])
            count += 1
    elif 11 < count < 23:
        for c, d in zip(range(12), range(16,27)):
            ax = fig.add_subplot(gs[1,c])
            sns.violinplot(x=df.iloc[:,[d]], ax=ax)
            ax.set_title(df.iloc[:,[d]].columns.values[0])
            count += 1
    elif 22 < count < 34:
        for c, d in zip(range(12), range(27,38)):
            ax = fig.add_subplot(gs[2,c])
            sns.violinplot(x=df.iloc[:,[d]], ax=ax)
            ax.set_title(df.iloc[:,[d]].columns.values[0])
            count += 1
    elif 33 < count < 45:
        for c, d in zip(range(12), range(38,49)):
            ax = fig.add_subplot(gs[3,c])
            sns.violinplot(x=df.iloc[:,[d]], ax=ax)
            ax.set_title(df.iloc[:,[d]].columns.values[0])
            count += 1
    elif 44 < count < 56:
        for c, d in zip(range(12), range(49,60)):
            ax = fig.add_subplot(gs[4,c])
            sns.violinplot(x=df.iloc[:,[d]], ax=ax)
            ax.set_title(df.iloc[:,[d]].columns.values[0])
            count += 1
    
fig.suptitle("WGM 2018 Data Distribution", fontsize=18, y=0.95)
fig.tight_layout()
fig.savefig('/Users/Ming/Documents/GitHub/wellcome_dissertation/WellcomePy/py_image/datadist-2.png')
plt.close()

## Missing Data Analysis

In [135]:
df = open_csv('full_data.csv')
missingdf = missing_val(df)
save_csv(missingdf, "missing_df.csv")

In [15]:
# Visualize missing values 
df = open_csv('full_data.csv')
fig, ax = plt.subplots(figsize=(20,20))
msplot = sns.heatmap(df.isnull(), cbar=False, ax=ax)
fig = msplot.get_figure()
fig.suptitle('Missing data in original WGM 2018 data set', y=0.94, fontsize=30)
fig.savefig('/Users/Ming/Documents/GitHub/wellcome_dissertation/WellcomePy/py_image/msplot.png')
plt.close()

In [482]:
# Check abnormal missing data for TRU_NATGOV
normal, mapnormal, notnormal, mapabnormal = check_ctry_unique(check_ctry('TRU_NATGOV'), 'TRU_NATGOV')
print(notnormal)
print(mapabnormal)

[52, 30, 2, 137, 53, 5, 185, 191, 193, 50]
['Cambodia', 'China', 'Egypt', 'Kuwait', 'Laos', 'Saudi Arabia', 'Tajikistan', 'Turkmenistan', 'United Arab Emirates', 'Vietnam']


Shows these countries have entire missing value for TRU_NATGOV

In [385]:
normal, mapnormal, notnormal, mapabnormal = check_ctry_unique(check_ctry('TRU_GOVADV'), 'TRU_GOVADV')
print(check_ctry('TRU_GOVADV'))
print(notnormal)
print(mapabnormal)

Country  Country Code
0  Saudi Arabia             5
[5]
['Saudi Arabia']


In [483]:
# Check abnormal missing data for RELIG
normal, mapnormal, notnormal, mapabnormal = check_ctry_unique(check_ctry('RELIG'), 'RELIG')
print(notnormal)
print(mapabnormal)

[30]
['China']


Shows that only China have entire missing value in RELIG

In [359]:
normal, mapnormal, notnormal, mapabnormal = check_ctry_unique(check_ctry('SC_DISRELIG'), 'SC_DISRELIG')
print(notnormal)
print(mapabnormal)

[30, 137, 5]
['China', 'Kuwait', 'Saudi Arabia']


In [358]:
normal, mapnormal, notnormal, mapabnormal = check_ctry_unique(check_ctry('BLV_SCRELIG'), 'BLV_SCRELIG')
print(notnormal)
print(mapabnormal)

[30, 137, 5]
['China', 'Kuwait', 'Saudi Arabia']


In [484]:
normal, mapnormal, notnormal, mapabnormal = check_ctry_unique(check_ctry('HHI'), 'HHI')
print(notnormal)
print(mapabnormal)

[62, 64, 32]
['Mauritania', 'Niger', 'Venezuela']


In [485]:
normal, mapnormal, notnormal, mapabnormal = check_ctry_unique(check_ctry('CTRY_INC'), 'CTRY_INC')
print(notnormal)
print(mapabnormal)

[202]
['Northern Cyprus']


## Exploratory Data Analysis

### Descriptive Statistics

In [ ]:
# Calculate response for these variables (plot in Tableau)
grp1 = ['SC_KNOWL', 'UND_SCSCI', 'SC_DZ', 'SC_POET', 'SC_PS', 'SC_SS', 'SC_UNI', 'SCINFO_30D', 'MDHINFO_30D', 'SC_INT', 'MDH_INT', 'SCI_BENPPL', 'SCI_BENRESP', 'SCTECH_IMPRLIFE', 'SCTECH_JOBS']
grp2 = ['TRU_NEIGHB', 'TRU_NATGOV', 'TRU_SCI', 'TRU_JO', 'TRU_HCW', 'TRU_NGOPPL', 'TRU_TH', 'TRU_SC', 'TRUSCI_ACCINFO', 'TRUSCIUNI_BEN', 'TRUSCIUNI_HON', 'TRUSCICOM_BEN', 'TRUSCICOM_HON', 'TRU_PPLADV', 'TRU_GOVADV', 'TRU_HCWADV', 'CONF_NGO', 'CONF_HOSP']
grp3 = ['HRD_VAC', 'VAC_IMPTCH', 'VAC_SF', 'VAC_EFF', 'ANY_CH', 'CH_RVAC']
grp4 = ['RELIG', 'SC_DISRELIG', 'BLV_SCRELIG']

In [ ]:
def rescount(list):
    df = open_csv('full_data.csv')
    grpdf = pd.DataFrame()
    for col in list:
        val = df[col].value_counts().sort_index(ascending=True).values
        idx = df[col].value_counts().sort_index(ascending=True).index
        var = [df[[col]].columns.values[0] for i in range(len(df[col].value_counts()))]
        d = {'var':var, 'val':val, 'idx':idx}
        grpdf = pd.concat([pd.DataFrame(d), grpdf])
    return grpdf

In [ ]:
grp1df = rescount(grp1)
grp2df = rescount(grp2)
grp3df = rescount(grp3)
grp4df = rescount(grp4)

In [ ]:
save_csv(grp1df, 'grp1df.csv')
save_csv(grp2df, 'grp2df.csv')
save_csv(grp3df, 'grp3df.csv')
save_csv(grp4df, 'grp4df.csv')

### Compare Vaccine Attitude (Child vs No Child)

In [228]:
df = open_csv('fpp_df.csv')
df_ch = open_csv('df_ch.csv')
df_noch = open_csv('df_noch.csv')

In [315]:
def check_vac_res(df, name, word):
    response = {0:1, 1:2, 2:3, 3:4, 4:5, 5:99}
    ndf_1 = df['VAC_IMPTCH'].value_counts().sort_index(ascending=True).values
    ndf_2 = df['VAC_SF'].value_counts().sort_index(ascending=True).values
    ndf_3 = df['VAC_EFF'].value_counts().sort_index(ascending=True).values
    fdf = pd.DataFrame(data=zip(ndf_1, ndf_2, ndf_3), columns=['VAC_IMPTCH', 'VAC_SF', 'VAC_EFF'])
    fdf = fdf.reset_index().rename(columns={'index':'Responses'})
    fdf['Responses'] = fdf.replace(response).astype('str')
    fdf['Child'] = [word for i in range(len(fdf))]
    return fdf

In [316]:
vacres_ch = check_vac_res(df_ch, 'df_ch', 'Yes')

In [317]:
vacres_noch = check_vac_res(df_noch, 'df_noch', 'No')

In [318]:
vacres = pd.concat([vacres_ch, vacres_noch])
vacres = vacres.reset_index(drop=True)

In [319]:
save_csv(vacres, 'vacres.csv')

### Compare Countries Vaccine Attitude Index

In [24]:
def drop_vac99(df):
    vdf = df[(df['VAC_IMPTCH'] != 99) & (df['VAC_SF'] != 99) & (df['VAC_EFF'] != 99)]
    return vdf

def decode_vac(df):
    vac_code = {1:5, 2:4, 3:3, 4:2, 5:1}
    df['VAC_IMPTCH'] = df['VAC_IMPTCH'].replace(vac_code)
    df['VAC_SF'] = df['VAC_SF'].replace(vac_code)
    df['VAC_EFF'] = df['VAC_EFF'].replace(vac_code)
    return df

def create_index(df):
    idx = df[['VAC_IMPTCH', 'VAC_SF', 'VAC_EFF']].mean(axis=1)
    df.insert(loc=36, column='VAC_IDX', value=idx)
    return df

def assign_vacidx_cat(df):
    idx_cat = []
    for idx in df['VAC_IDX']:
        if idx < 1.7:
            idx = 'Low'
            idx_cat.append(idx)
        elif 1.7 < idx < 3.4:
            idx = 'Medium'
            idx_cat.append(idx)
        else:
            idx = 'High'
            idx_cat.append(idx)
    df.insert(loc=37, column='VAC_LVL', value=idx_cat)
    return df

In [7]:
df = open_csv('fpp_df.csv')
ftrain_ch = open_csv('ftrain_ch.csv')
ftest_ch = open_csv('ftest_ch.csv')
ftrain_noch = open_csv('ftrain_noch.csv')
ftest_noch = open_csv('ftest_noch.csv')

In [497]:
# Drop response = 99 in vaccine questions
df_vacidx = drop_vac99(df)
train_ch_vacidx = drop_vac99(ftrain_ch)
test_ch_vacidx = drop_vac99(ftest_ch)
train_noch_vacidx = drop_vac99(ftrain_noch)
test_noch_vacidx =drop_vac99(ftest_noch)

In [498]:
# Decode vaccine questions: 1=5, 2=4, 3=3, 4=2, 5=1
df_vacidx = decode_vac(df_vacidx)
train_ch_vacidx = decode_vac(train_ch_vacidx)
test_ch_vacidx = decode_vac(test_ch_vacidx)
train_noch_vacidx = decode_vac(train_noch_vacidx)
test_noch_vacidx =decode_vac(test_noch_vacidx)

In [499]:
# Create Vaccine Index
df_vacidx = create_index(df_vacidx)
train_ch_vacidx = create_index(train_ch_vacidx)
test_ch_vacidx = create_index(test_ch_vacidx)
train_noch_vacidx = create_index(train_noch_vacidx)
test_noch_vacidx = create_index(test_noch_vacidx)

In [500]:
# Assign Vaccine Index into Low, Medium, High Category
df_vacidx = assign_vacidx_cat(df_vacidx)
train_ch_vacidx = assign_vacidx_cat(train_ch_vacidx)
test_ch_vacidx = assign_vacidx_cat(test_ch_vacidx)
train_noch_vacidx = assign_vacidx_cat(train_noch_vacidx)
test_noch_vacidx = assign_vacidx_cat(test_noch_vacidx)

In [501]:
save_csv(df_vacidx, 'df_vacidx.csv')
save_csv(train_ch_vacidx, 'train_ch_vacidx.csv')
save_csv(test_ch_vacidx, 'test_ch_vacidx.csv')
save_csv(train_noch_vacidx, 'train_noch_vacidx.csv')
save_csv(test_noch_vacidx, 'test_noch_vacidx.csv')

In [514]:
ctry = df_vacidx['CTRY'].replace(country)
df_vacidx_ctry = df_vacidx.copy()
df_vacidx_ctry.insert(loc=1, column='CTRY_NAME', value=ctry)

In [518]:
save_csv(df_vacidx_ctry, 'df_vacidx_ctry.csv')

### Compare Vaccine Attitude with Demographic and Country-level variables

In [111]:
df = open_csv('full_data.csv')

In [6]:
df_99 = drop_vac99(df)

In [174]:
# Without NA
def ridgedist(df):
    age_cat = {1:'Adolescent', 2:'Adult', 3:'Elder', np.NaN:'NAN'}
    gen = {1:'Male', 2:'Female'}
    edu = {1:'Primary', 2:'Secondary', 3:'Tertiary', np.NaN:'NAN'}
    liv_ar = {1:'Rural', 2:'Urban', np.NaN:'NAN'}
    hhi = {1:'Lowest', 2:'Lower Middle', 3:'Middle', 4:'Upper Middle', 5:'Upper', np.NaN:'NAN'}
    # rgn = {0:'N/A', 1:'Eastern Africa',2:'Central Africa',3:'North Africa',4:'Southern Africa',5:'Western Africa',6:'Central America and Mexico',7:'Northern America',8:'South America',9:'Central Asia',10:'East Asia',11:'Southeast Asia',12:'South Asia',13:'Middle East',14:'Eastern Europe',15:'Northern Europe',16:'Southern Europe',17:'Western Europe',18:'Aus/NZ'}
    rgn = {0:'N/A', 1:'EAFR',2:'CAFR',3:'NAFR',4:'SAFR',5:'WAFR',6:'CAMX',7:'NAM',8:'SAM',9:'CCA',10:'EA',11:'SEA',12:'SA',13:'ME',14:'EE',15:'NE',16:'SE',17:'WE',18:'AUS/NZ'}
    subj_hhi = {1:'Comfortable', 2:'Sustainable', 3:'Difficult', np.NaN:'NAN'}
    ctry_inc = {1:'LI', 2:'LMI', 3:'UMI', 4:'HI', np.NaN:'NAN'}
    emp_stat = {1:'F/T Employer', 2:'F/T Self', 3:'P/T - F/T', 4:'UE', 5:'P/T + F/T', 6:'OW', np.NaN:'NAN'}
    df_99 = drop_vac99(df)
    df_99['AGE_CAT'] = df_99['AGE_CAT'].replace(age_cat)
    df_99['GEN'] = df_99['GEN'].replace(gen)
    df_99['EDU'] = df_99['EDU'].replace(edu)
    df_99['LIV_AR'] = df_99['LIV_AR'].replace(liv_ar)
    df_99['HHI'] = df_99['HHI'].replace(hhi)
    df_99['RGN'] = df_99['RGN'].replace(rgn)
    df_99['SUBJ_HHI'] = df_99['SUBJ_HHI'].replace(subj_hhi)
    df_99['CTRY_INC'] = df_99['CTRY_INC'].replace(ctry_inc)
    df_99['EMP_STAT'] = df_99['EMP_STAT'].replace(emp_stat)
    return df_99

In [175]:
ridgedist_df = ridgedist(df)
save_csv(ridgedist_df, 'ridgedist_df.csv')

In [22]:
# With NA
def ridgedist_na_ch(df):
    age_cat = {1:'Adolescent', 2:'Adult', 3:'Elder'}
    gen = {1:'Male', 2:'Female'}
    edu = {1:'Primary', 2:'Secondary', 3:'Tertiary'}
    liv_ar = {1:'Rural', 2:'Urban'}
    hhi = {1:'Lowest', 2:'Lower Middle', 3:'Middle', 4:'Upper Middle', 5:'Upper'}
    # rgn = {0:'N/A', 1:'Eastern Africa',2:'Central Africa',3:'North Africa',4:'Southern Africa',5:'Western Africa',6:'Central America and Mexico',7:'Northern America',8:'South America',9:'Central Asia',10:'East Asia',11:'Southeast Asia',12:'South Asia',13:'Middle East',14:'Eastern Europe',15:'Northern Europe',16:'Southern Europe',17:'Western Europe',18:'Aus/NZ'}
    rgn = {0:'N/A', 1:'EAFR',2:'CAFR',3:'NAFR',4:'SAFR',5:'WAFR',6:'CAMX',7:'NAM',8:'SAM',9:'CCA',10:'EA',11:'SEA',12:'SA',13:'ME',14:'EE',15:'NE',16:'SE',17:'WE',18:'AUS/NZ'}
    subj_hhi = {1:'Comfortable', 2:'Sustainable', 3:'Difficult'}
    ctry_inc = {1:'LI', 2:'LMI', 3:'UMI', 4:'HI'}
    emp_stat = {1:'F/T Employer', 2:'F/T Self', 3:'P/T - F/T', 4:'UE', 5:'P/T + F/T', 6:'OW'}
    any_ch = {1:'HV_CH', 2:'NO_CH', 3:'HV_CH_NOLIV', 4:'DK', 5:'REF'}
    ch_rvac = {1:'Yes', 2:'No', 98:'DK', 99:'RF'}
    df_99 = drop_vac99(df)
    df_99['AGE_CAT'] = df_99['AGE_CAT'].replace(age_cat)
    df_99['GEN'] = df_99['GEN'].replace(gen)
    df_99['EDU'] = df_99['EDU'].replace(edu)
    df_99['LIV_AR'] = df_99['LIV_AR'].replace(liv_ar)
    df_99['HHI'] = df_99['HHI'].replace(hhi)
    df_99['RGN'] = df_99['RGN'].replace(rgn)
    df_99['SUBJ_HHI'] = df_99['SUBJ_HHI'].replace(subj_hhi)
    df_99['CTRY_INC'] = df_99['CTRY_INC'].replace(ctry_inc)
    df_99['EMP_STAT'] = df_99['EMP_STAT'].replace(emp_stat)
    df_99['ANY_CH'] = df_99['ANY_CH'].replace(any_ch)
    df_99['CH_RVAC'] = df_99['CH_RVAC'].replace(ch_rvac)
    return df_99

In [25]:
ridgedist_na_ch_df = ridgedist_na_ch(df)
save_csv(ridgedist_na_ch_df, 'ridgedist_na_ch_df.csv')

In [115]:
# Without drop_99
def vac_overview(df_99):
    age_cat = {1:'Adolescent', 2:'Adult', 3:'Elder'}
    gen = {1:'Male', 2:'Female'}
    edu = {1:'Primary', 2:'Secondary', 3:'Tertiary'}
    liv_ar = {1:'Rural', 2:'Urban'}
    hhi = {1:'Lowest', 2:'Lower Middle', 3:'Middle', 4:'Upper Middle', 5:'Upper'}
    rgn = {0:'N/A', 1:'Eastern Africa',2:'Central Africa',3:'North Africa',4:'Southern Africa',5:'Western Africa',6:'Central America & Mexico',7:'Northern America',8:'South America',9:'Central Asia',10:'East Asia',11:'Southeast Asia',12:'South Asia',13:'Middle East',14:'Eastern Europe',15:'Northern Europe',16:'Southern Europe',17:'Western Europe',18:'Australia & NZ'}
    subj_hhi = {1:'Comfortable', 2:'Sustainable', 3:'Difficult'}
    ctry_inc = {1:'LI', 2:'LMI', 3:'UMI', 4:'HI'}
    # emp_stat = {1:'F/T Employer', 2:'F/T Self', 3:'P/T - F/T', 4:'UE', 5:'P/T + F/T', 6:'OW'}
    emp_stat = {1:'Employed full time for an employer', 2:'Employed full time for self', 3:'Employed part time but do not want full time', 4:'Unemployed', 5:'Employed part time but want full time', 6:'Out of workforce'}
    any_ch = {1:'HV_CH', 2:'NO_CH', 3:'HV_CH_NOLIV', 4:'DK', 5:'REF'}
    ch_rvac = {1:'Yes', 2:'No', 98:'DK', 99:'RF'}
    df_99['AGE_CAT'] = df_99['AGE_CAT'].replace(age_cat)
    df_99['GEN'] = df_99['GEN'].replace(gen)
    df_99['EDU'] = df_99['EDU'].replace(edu)
    df_99['LIV_AR'] = df_99['LIV_AR'].replace(liv_ar)
    df_99['HHI'] = df_99['HHI'].replace(hhi)
    df_99['RGN'] = df_99['RGN'].replace(rgn)
    df_99['SUBJ_HHI'] = df_99['SUBJ_HHI'].replace(subj_hhi)
    df_99['CTRY_INC'] = df_99['CTRY_INC'].replace(ctry_inc)
    df_99['EMP_STAT'] = df_99['EMP_STAT'].replace(emp_stat)
    df_99['ANY_CH'] = df_99['ANY_CH'].replace(any_ch)
    df_99['CH_RVAC'] = df_99['CH_RVAC'].replace(ch_rvac)
    return df_99

In [116]:
df = open_csv('full_data.csv')
vac_overview = vac_overview(df)
save_csv(vac_overview, 'vac_overview.csv')

### Include decoded ANY_CH and CH_RVAC

In [13]:
any_ch = {1:'HV_CH', 2:'NO_CH', 3:'HV_CH_NOLIV', 4:'DK', 5:'REF', np.NaN:'NAN'}
ch_rvac = {1:'Yes', 2:'No', 98:'DK', 99:'RF', np.NaN:'NAN'}
ridgedist_df = open_csv('ridgedist_df.csv')
ridgedistvac_df = ridgedist_df.copy()
ridgedistvac_df['ANY_CH'] = ridgedistch_df['ANY_CH'].replace(any_ch)
ridgedistvac_df['CH_RVAC'] = ridgedistch_df['CH_RVAC'].replace(ch_rvac)
save_csv(ridgedistvac_df, 'ridgedistvac_df.csv')

### Relationship between TRUSCI_IDX and Vaccine Attitudes

In [199]:
recode = {1:'Strongly agree', 2:'Somewhat agree', 3:'Neither agree nor disagree', 4:'Somewhat disagree', 5:'Strongly disagree', 99:"DK/Refused"}
trusci_df = df.copy()
trusci_df['VAC_IMPTCH'] = trusci_df['VAC_IMPTCH'].replace(recode)
trusci_df['VAC_SF'] = trusci_df['VAC_SF'].replace(recode)
trusci_df['VAC_EFF'] = trusci_df['VAC_EFF'].replace(recode)
fig, ax = plt.subplots(1,3, figsize=(40,14))
ax1 = sns.boxplot(x='VAC_IMPTCH', y='TRUSCI_IDX', data=trusci_df, ax=ax[0], order=['Strongly agree', 'Somewhat agree', 'Neither agree nor disagree', 'Somewhat agree', 'Strongly disagree', 'DK/Refused'])
ax2 = sns.boxplot(x='VAC_SF', y='TRUSCI_IDX', data=trusci_df, ax=ax[1], order=['Strongly agree', 'Somewhat agree', 'Neither agree nor disagree', 'Somewhat agree', 'Strongly disagree', 'DK/Refused'])
ax3 = sns.boxplot(x='VAC_EFF', y='TRUSCI_IDX', data=trusci_df, ax=ax[2], order=['Strongly agree', 'Somewhat agree', 'Neither agree nor disagree', 'Somewhat agree', 'Strongly disagree', 'DK/Refused'])
plt.setp(ax1.get_xticklabels(), rotation=25)
plt.setp(ax2.get_xticklabels(), rotation=25)
plt.setp(ax3.get_xticklabels(), rotation=25)
ax1.set_title("Vaccine is important to children", fontsize=16)
ax2.set_title("Vaccine is safe", fontsize=16)
ax3.set_title("Vaccine is effective", fontsize=16)
plt.suptitle("Relationships between Trust Scientist Index and Vaccine Attitudes", fontsize=20)
fig.savefig("/Users/Ming/Documents/GitHub/wellcome_dissertation/WellcomePy/py_image/trusc_idx_vac.png")
plt.close()

### Check country percentage of never heard of vaccine

In [ ]:
df = open_csv("vac_overview.csv")
d = {}
for i, k in country.items():
    try:
        d[i] = [len(df[df['CTRY']==i]),df[df['CTRY']==i]['HRD_VAC'].value_counts()[2]]
    except:
        pass
ctry_hrdvac = pd.DataFrame(d).transpose().reset_index()
ctry_hrdvac = ctry_hrdvac.rename(columns={'index':'Country', 0:'Total', 1:'Never Heard Vaccine'})
ctry_hrdvac['Country'] = ctry_hrdvac['Country'].replace(country)
save_csv(ctry_hrdvac, 'ctry_hrdvac.csv')

### Check difference in attitude within each region

In [98]:
rgn = {0:'N/A', 1:'Eastern Africa',2:'Central Africa',3:'North Africa',4:'Southern Africa',5:'Western Africa',6:'Central America and Mexico',7:'Northern America',8:'South America',9:'Central Asia',10:'East Asia',11:'Southeast Asia',12:'South Asia',13:'Middle East',14:'Eastern Europe',15:'Northern Europe',16:'Southern Europe',17:'Western Europe',18:'Aus/NZ'}

In [197]:
df = open_csv("vac_overview.csv")
d = {}
for i, k in rgn.items():
    try:
        d[k] = [df[df['RGN']== k]['VAC_IMPTCH'].count(),df[df['RGN']==k]['VAC_IMPTCH'].value_counts().sort_index(ascending=True)[[1,2]].sum(), df[df['RGN']==k]['VAC_SF'].value_counts().sort_index(ascending=True)[[1,2]].sum(), df[df['RGN']==k]['VAC_EFF'].value_counts().sort_index(ascending=True)[[1,2]].sum()]
    except:
        pass

rgn_vac = pd.DataFrame(d).transpose().reset_index()
rgn_vac = rgn_vac.rename(columns={'index':'Country', 0:'Total', 1:'VAC_IMPTCH', 2:'VAC_SF', 3:'VAC_EFF'})
save_csv(rgn_vac, 'rgn_vac.csv')

In [210]:
# Check discrepency of response (strongly agree/somewhat agree) %
rgn_vac['VAC_IMPTCH_SF'] = (((rgn_vac['VAC_IMPTCH']/rgn_vac['Total']) - (rgn_vac['VAC_SF']/rgn_vac['Total']))*100).round(0).astype('int64')
rgn_vac['VAC_IMPTCH_EFF'] = (((rgn_vac['VAC_IMPTCH']/rgn_vac['Total']) - (rgn_vac['VAC_EFF']/rgn_vac['Total']))*100).round(0).astype('int64')
rgn_vac['VAC_SF_EFF'] = (((rgn_vac['VAC_SF']/rgn_vac['Total']) - (rgn_vac['VAC_EFF']/rgn_vac['Total']))*100).round(0).astype('int64')

### Check attitudes (disagree) for every country 

In [274]:
df = open_csv('full_data.csv')
d = {}
for i, k in country.items():
    try:
        d[i] = [df[df['CTRY']== i]['VAC_IMPTCH'].count(), df[df['CTRY']==i]['VAC_IMPTCH'].value_counts().sort_index(ascending=True)[[4,5]].sum(), df[df['CTRY']==i]['VAC_SF'].value_counts().sort_index(ascending=True)[[4,5]].sum(), df[df['CTRY']==i]['VAC_EFF'].value_counts().sort_index(ascending=True)[[4,5]].sum()]
    except:
        pass

ctry_disagree_vac = pd.DataFrame(d).transpose().reset_index()
ctry_disagree_vac = ctry_disagree_vac.rename(columns={'index':'Country', 0:'Total', 1:'VAC_IMPTCH', 2:'VAC_SF', 3:'VAC_EFF'})

In [278]:
ctry_disagree_vac.insert(loc=1, column='Country Name', value=ctry_disagree_vac['Country'].replace(country))

In [280]:
save_csv(ctry_disagree_vac, 'ctry_disagree_vac.csv')

### Check Age Distribution for Vaccine Questions

In [ ]:
vac_age = open_csv("vac_overview.csv")
vac = {1:"Strongly agree", 2:'Somewhat agree', 3:'Neutral', 4:'Somewhat disagree', 5:'Strongly disagree', 99:'No Opinion'}
vac_age['VAC_IMPTCH'] = vac_age['VAC_IMPTCH'].replace(vac)
vac_age['VAC_SF'] = vac_age['VAC_SF'].replace(vac)
vac_age['VAC_EFF'] = vac_age['VAC_EFF'].replace(vac)
save_csv(vac_age, 'vac_age.csv')

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(30,8))
sns.violinplot(x='VAC_IMPTCH', y='AGE', data=vac_age, ax=ax[0], order=['Strongly agree', 'Somewhat agree', 'Neutral', 'Somewhat disagree', 'Strongly disagree', 'No Opinion'])
ax[0].set_title('Vaccine is important to children')
sns.violinplot(x='VAC_SF', y='AGE', data=vac_age, ax=ax[1], order=['Strongly agree', 'Somewhat agree', 'Neutral', 'Somewhat disagree', 'Strongly disagree', 'No Opinion'])
ax[1].set_title('Vaccine is safe')
sns.violinplot(x='VAC_EFF', y='AGE', data=vac_age, ax=ax[2], order=['Strongly agree', 'Somewhat agree', 'Neutral', 'Somewhat disagree', 'Strongly disagree', 'No Opinion'])
ax[2].set_title('Vaccine is effective')
plt.setp(ax[0].get_xticklabels(), rotation=25)
plt.setp(ax[1].get_xticklabels(), rotation=25)
plt.setp(ax[2].get_xticklabels(), rotation=25)
plt.suptitle("Distribution of Age for All Responses in Vaccine Questions", fontsize=20)
fig.savefig("/Users/Ming/Documents/GitHub/wellcome_dissertation/WellcomePy/py_image/vac_age.png")
plt.close()

## Index Development 

In [ ]:
# Index development (Use full dataset)
df = open_csv('full_data.csv')
df = df.loc[:, ['VAC_IMPTCH', 'VAC_SF', 'VAC_EFF']]
df = df.dropna(subset=['VAC_IMPTCH', 'VAC_SF', 'VAC_EFF'], how='all')
df = df[(df['VAC_IMPTCH'] != 99) & (df['VAC_SF'] != 99) & (df['VAC_EFF'] != 99)]

# Factor Analysis
# Insert prepared dataframe (must not contain single unique value/Null in each column), number of factors, and loads
loads, ev, v = factor_analysis(df, 1)
factor_dataframe(df, loads)
factor_plot(df, ev, 'vacidx.png')

# Cronbach Alpha test
factor1 = df[['VAC_IMPTCH', 'VAC_SF', 'VAC_EFF']]
factor1_alpha = pg.cronbach_alpha(factor1)
print(factor1_alpha)

All loadings > 0.5, and Cronbach are 0.76 which is reliable to create index. Hence, take simple average of these variables.

In [174]:
ftrain_ch = open_csv('ftrain_ch.csv')
ftest_ch = open_csv('ftest_ch.csv')
ftrain_noch = open_csv('ftrain_noch.csv')
ftest_noch = open_csv('ftest_noch.csv')

In [188]:
idx_dev(ftrain_ch)



            Factor 1
VAR                 
VAC_EFF     0.781782
VAC_SF      0.765461
VAC_IMPTCH  0.683318
(0.7773013268916602, array([0.774, 0.78 ]))


In [189]:
idx_dev(ftest_ch)



            Factor 1
VAR                 
VAC_EFF     0.775087
VAC_SF      0.760399
VAC_IMPTCH  0.667029
(0.7667380783240179, array([0.761, 0.773]))


In [190]:
idx_dev(ftrain_noch)



            Factor 1
VAR                 
VAC_SF      0.737136
VAC_EFF     0.721709
VAC_IMPTCH  0.664569
(0.7419935174277057, array([0.736, 0.747]))


In [191]:
idx_dev(ftest_noch)



            Factor 1
VAR                 
VAC_SF      0.754230
VAC_EFF     0.735574
VAC_IMPTCH  0.682700
(0.7594353878939415, array([0.749, 0.77 ]))


In [70]:
from factor_analyzer import FactorAnalyzer
import pingouin as pg

def idx_dev(df):
    df = df.loc[:, ['VAC_IMPTCH', 'VAC_SF', 'VAC_EFF']]
    df = df[(df['VAC_IMPTCH'] != 99) & (df['VAC_SF'] != 99) & (df['VAC_EFF'] != 99)]
    # Factor Analysis
    # Insert prepared dataframe (must not contain single unique value/null in each column) and number of factor(s)
    loads, ev, v = factor_analysis(df, 1)
    print("=== Loadings of Each Variable ===")
    factor_dataframe(df, loads.round(2))
    print("\n")

    # Cronbach Alpha test
    factor = df[['VAC_IMPTCH', 'VAC_SF', 'VAC_EFF']]
    factor_alpha = pg.cronbach_alpha(factor)
    print("=== Cronbach Alpha ===")
    print(factor_alpha[0].round(2))

In [71]:
df = open_csv("full_data.csv")
idx_dev(df)

=== Loadings of Each Variable ===


            Factor 1
VAR                 
VAC_EFF         0.78
VAC_SF          0.77
VAC_IMPTCH      0.68


=== Cronbach Alpha ===
0.77


## Test Code

In [ ]:
# Dummy df for testing MICE imputation in R
from random import randint, choice
d1 = []
random.seed(42)
for i in range(100):
    d1.append(choice([randint(0,1), randint(97,99)]))
d2 = []
random.seed(60)
for i in range(100):
    d2.append(choice([randint(0,1), randint(97,99)]))
d3 = []
random.seed(100)
for i in range(100):
    d3.append(choice([randint(0,1), randint(97,99)]))
edudf = pd.DataFrame(data=zip(d1,d2,d3), columns=['PS', 'SS', 'UNI'])
edudf.loc[edudf['PS'] == 0, 'SS'] = np.NaN
edudf.loc[edudf['SS'] == 0, 'UNI'] = np.NaN
edudf = edudf.astype('Int64')
edudf.to_csv('/Users/Ming/Documents/GitHub/wellcome_dissertation/WellcomePy/csv_data/edudf.csv', index=False)

In [ ]:
test_ch = open_csv("test_ch.csv")
imptrain = open_csv("imptrain.csv")

In [ ]:
mfa_test = test_ch
mfa_test[['SC_SS', 'SC_UNI', 'VAC_IMPTCH', 'VAC_SF', 'VAC_EFF', 'CH_RVAC', 'AGE_CAT']] = mfa_test[['SC_SS', 'SC_UNI', 'VAC_IMPTCH', 'VAC_SF', 'VAC_EFF', 'CH_RVAC', 'AGE_CAT']].astype('int64')

In [ ]:
mfa_test = mfa_test[['CTRY', 'HRD_VAC', 'VAC_IMPTCH', 'VAC_SF', 'VAC_EFF', 'ANY_CH', 'CH_RVAC', 'AGE', 'AGE_CAT', 'GEN']]
country = ['CTRY']
demographic = ['AGE', 'AGE_CAT', 'GEN']
vaccine = ['HRD_VAC', 'VAC_IMPTCH', 'VAC_SF', 'VAC_EFF', 'ANY_CH', 'CH_RVAC']
new_cols = country + demographic + vaccine
mfa_test = mfa_test[new_cols]
save_csv(mfa_test, 'mfa_test.csv')

## Modelling

In [3]:
# Open data
ftrain_ch = open_csv('train_ch_vacidx.csv')
ftest_ch = open_csv('test_ch_vacidx.csv')
ftrain_noch = open_csv('train_noch_vacidx.csv')
ftest_noch = open_csv('test_noch_vacidx.csv')

In [4]:
# For data with child
def model_preprocess_clf_ch(df):
    recode = {'High':3, 'Medium':2, 'Low':1}
    df['VAC_LVL'] = df['VAC_LVL'].replace(recode)
    df = df.drop(['CTRY', 'VAC_IDX', 'HRD_VAC', 'VAC_IMPTCH', 'VAC_SF', 'VAC_EFF', 'ANY_CH'], axis=1)
    label = df[['VAC_LVL']]
    feature = df.drop('VAC_LVL', axis=1)
    return (feature, label)

# For data without child
def model_preprocess_clf_noch(df):
    recode = {'High':3, 'Medium':2, 'Low':1}
    df['VAC_LVL'] = df['VAC_LVL'].replace(recode)
    df = df.drop([ 'CTRY', 'VAC_IDX', 'HRD_VAC', 'VAC_IMPTCH', 'VAC_SF', 'VAC_EFF'], axis=1)
    label = df[['VAC_LVL']]
    feature = df.drop('VAC_LVL', axis=1)
    return (feature, label)

In [5]:
# Preprocess for modelling (split feature and label)
ch_train_feature, ch_train_label= model_preprocess_clf_ch(ftrain_ch)
ch_test_feature, ch_test_label = model_preprocess_clf_ch(ftest_ch)
noch_train_feature, noch_train_label = model_preprocess_clf_noch(ftrain_noch)
noch_test_feature, noch_test_label = model_preprocess_clf_noch(ftest_noch)

In [67]:
# Create full dataset for child and no child (for CV)
chdf_feature = pd.concat([ch_train_feature, ch_test_feature]).reset_index(drop=True)
chdf_label = pd.concat([ch_train_label, ch_test_label]).reset_index(drop=True)
nochdf_feature = pd.concat([noch_train_feature, noch_test_feature]).reset_index(drop=True)
nochdf_label = pd.concat([noch_train_label, noch_test_label]).reset_index(drop=True)

### Test CV

In [146]:
def tree_cv(X_train, y_train, mod, mod_name):
    cvs = cross_val_score(mod, X_train, y_train, cv=5, scoring='accuracy')
    print(cvs)
    print('\n')
    print("=== Mean Accuracy Score ===")
    print("Mean Accuracy Score - %s"  %mod_name + ":", cvs.mean().round(2))
    return cvs

In [148]:
def grid_search(mod, param, X_train, y_train):
    gs = GridSearchCV(estimator=mod, param_grid=param, cv=5, scoring='accuracy')
    gs.fit(X_train, y_train)
    print(gs.best_params_)

In [151]:
# Random Forest CV
rfc = RandomForestClassifier()
rfc.fit(ch_train_feature, ch_train_label)
rfc_cvs_ori = tree_cv(ch_train_feature, ch_train_label, rfc, 'Random Forest')

[0.88785249 0.88763557 0.8879248  0.88785249 0.8897968 ]


=== Mean Accuracy Score ===
Mean Accuracy Score - Random Forest: 0.89


In [149]:
# Perform Grid Search
param = {'n_estimators':[110, 130, 150], 'max_depth':[20, 60, 100], 'oob_score':[True, False]}
grid_search(rfc, param, ch_train_feature, ch_train_label)

{'max_depth': 100, 'n_estimators': 130, 'oob_score': False}


In [150]:
# Random Forest CV using GS parameters
rfc = RandomForestClassifier(n_estimators=130, max_depth=100, oob_score=False)
rfc.fit(ch_train_feature, ch_train_label)
rfc_cvs = tree_cv(ch_train_feature, ch_train_label, rfc, 'Random Forest')

[0.88821403 0.88806941 0.88864787 0.88785249 0.88943524]


=== Mean Accuracy Score ===
Mean Accuracy Score - Random Forest: 0.89


In [ ]:
ht_data = {'Model':['Ori RF', 'GSCV RF'], 'Accuracy':[rfc_cvs_ori.mean().round(5), rfc_cvs.mean().round(5)]}
ht = pd.DataFrame(ht_data)
ht

In [171]:
# Plot before and after grid search
fig, ax = plt.subplots(figsize=(10,10))
sns.lineplot(x='Model', y='Accuracy', data=ht, sort=False, ax=ax)
fig.suptitle('RF Accuracy Before and After Hyperparameter Tuning')
fig.savefig("/Users/Ming/Documents/GitHub/wellcome_dissertation/WellcomePy/py_image/htrf.png")
plt.close()

### Compare RF, DF, and GB (Child)

In [8]:
dtc = DecisionTreeClassifier(random_state=34)
dtc.fit(ch_train_feature, ch_train_label)
pred = dtc.predict(ch_test_feature)
dtc_acc = accuracy_score(ch_test_label, pred)

In [9]:
rfc = RandomForestClassifier(random_state=12)
rfc.fit(ch_train_feature, ch_train_label)
pred = rfc.predict(ch_test_feature)
rfc_acc = accuracy_score(ch_test_label, pred)

In [6]:
gbc = GradientBoostingClassifier(random_state=10)
gbc.fit(ch_train_feature, ch_train_label)
pred = gbc.predict(ch_test_feature)
gbc_acc = accuracy_score(ch_test_label, pred)

In [10]:
def feature_importance(mod, df):
    # Plot feature importance
    feature_importance = mod.feature_importances_ # unsorted array
    idx = np.argsort(-feature_importance) # get sorted array indices in descending order (start with biggest value)

    # Create dataframe with feature and score using the sorted indices
    df_imp = pd.DataFrame(dict(Features=df.columns[idx],
                            Scores=feature_importance[idx]))
    fig, ax = plt.subplots(figsize=(10,10))
    ax = sns.barplot(x='Scores', y='Features', data=df_imp, palette='summer')
    ax.set_title('Variable Importance')

In [7]:
def feature_importance_dfonly(mod, df):
    # Plot feature importance
    feature_importance = mod.feature_importances_ # unsorted array
    idx = np.argsort(-feature_importance) # get sorted array indices in descending order (start with biggest value)

    # Create dataframe with feature and score using the sorted indices
    df_imp = pd.DataFrame(dict(Features=df.columns[idx],
                            Scores=feature_importance[idx]))
    return df_imp

In [11]:
dtc_df = feature_importance_dfonly(dtc, ch_train_feature)
rfc_df = feature_importance_dfonly(rfc, ch_train_feature)
gbc_df = feature_importance_dfonly(gbc, ch_train_feature)

In [78]:
# Plot Feature Importance
fig, ax = plt.subplots(1,3,figsize=(36,10))
sns.barplot(x='Scores', y='Features', data=dtc_df, palette='Blues_r', ax=ax[0])
ax[0].set_title('Decision Tree')
sns.barplot(x='Scores', y='Features', data=rfc_df, palette='Blues_r', ax=ax[1])
ax[1].set_title('Random Forest')
sns.barplot(x='Scores', y='Features', data=gbc_df, palette='Blues_r', ax=ax[2])
ax[2].set_title('Gradient Boosted Tree')
fig.suptitle('Significant Predictors for Vaccine Attitude (with child)', fontsize=24)
fig.savefig('/Users/Ming/Documents/GitHub/wellcome_dissertation/WellcomePy/py_image/fimp_child_new.png')
plt.close()

In [28]:
# Create df for comparison
comp_mod_ch = {'Model':['DT', 'RF', 'GB'], 'Accuracy':[dtc_acc.round(5), rfc_acc.round(5), gbc_acc.round(5)]}
comp_mod_df_ch = pd.DataFrame(comp_mod_ch)
save_csv(comp_mod_df_ch, 'ch_mod_comp.csv')

In [248]:
# Plot comparison
fig, ax = plt.subplots(figsize=(8,8))
sns.lineplot(x='Model', y='Accuracy', data=comp_mod_df_ch, sort=False, ax=ax)
fig.suptitle('Comparison of DT, RF, and GB accuracy (with child)')
fig.savefig('/Users/Ming/Documents/GitHub/wellcome_dissertation/WellcomePy/py_image/comp_child.png')
plt.close()

In [15]:
# Plot one feature importance only
fig, ax = plt.subplots(figsize=(14,14))
sns.barplot(x='Scores', y='Features', data=gbc_df, palette='Blues_r')
fig.suptitle('Significant Predictors from Gradient Boosting (With Child)', fontsize=26)
fig.savefig('/Users/Ming/Documents/GitHub/wellcome_dissertation/WellcomePy/py_image/fimp_ch_gbc.png')
plt.close()

### Compare DT, RF, and GB (without child)

In [16]:
dtc_noch = DecisionTreeClassifier(random_state=10)
dtc_noch.fit(noch_train_feature, noch_train_label)
pred = dtc_noch.predict(noch_test_feature)
noch_dtc_acc = accuracy_score(noch_test_label, pred)

In [17]:
rfc_noch = RandomForestClassifier(random_state=10)
rfc_noch.fit(noch_train_feature, noch_train_label)
pred = rfc_noch.predict(noch_test_feature)
noch_rfc_acc = accuracy_score(noch_test_label, pred)

In [18]:
gbc_noch = GradientBoostingClassifier(random_state=10)
gbc_noch.fit(noch_train_feature, noch_train_label)
pred = gbc_noch.predict(noch_test_feature)
noch_gbc_acc = accuracy_score(noch_test_label, pred)

In [19]:
noch_dtc_df = feature_importance_dfonly(dtc_noch, noch_train_feature)
noch_rfc_df = feature_importance_dfonly(rfc_noch, noch_train_feature)
noch_gbc_df = feature_importance_dfonly(gbc_noch, noch_train_feature)

In [45]:
# Plot Feature Importance
fig, ax = plt.subplots(1,3,figsize=(36,10))
sns.barplot(x='Scores', y='Features', data=noch_dtc_df, palette='Blues_r', ax=ax[0])
ax[0].set_title('Decision Tree')
sns.barplot(x='Scores', y='Features', data=noch_rfc_df, palette='Blues_r', ax=ax[1])
ax[1].set_title('Random Forest')
sns.barplot(x='Scores', y='Features', data=noch_gbc_df, palette='Blues_r', ax=ax[2])
ax[2].set_title('Gradient Boosted Tree')
fig.suptitle('Significant Predictors for Vaccine Attitude (without child)', fontsize=24)
fig.savefig('/Users/Ming/Documents/GitHub/wellcome_dissertation/WellcomePy/py_image/fimp_no_child_new.png')
plt.close()

In [29]:
# Create df for comparison
comp_mod_noch = {'Model':['DT', 'RF', 'GB'], 'Accuracy':[noch_dtc_acc.round(5), noch_rfc_acc.round(5), noch_gbc_acc.round(5)]}
comp_mod_df_noch = pd.DataFrame(comp_mod_noch)
save_csv(comp_mod_df_noch, 'noch_mod_comp.csv')

In [12]:
# Plot comparison
fig, ax = plt.subplots(figsize=(8,8))
sns.lineplot(x='Model', y='Accuracy', data=comp_mod_df_noch, sort=False, ax=ax)
fig.suptitle('Comparison of DT, RF, and GB accuracy (without child)')
fig.savefig('/Users/Ming/Documents/GitHub/wellcome_dissertation/WellcomePy/py_image/comp_nochild.png')
plt.close()

In [22]:
# Plot one feature importance only
fig, ax = plt.subplots(figsize=(14,14))
sns.barplot(x='Scores', y='Features', data=noch_gbc_df, palette='Blues_r')
fig.suptitle('Significant Predictors from Gradient Boosting (Without Child)', fontsize=26)
fig.savefig('/Users/Ming/Documents/GitHub/wellcome_dissertation/WellcomePy/py_image/fimp_noch_gbc.png')
plt.close()

In [27]:
# Combine GBC
fig, ax = plt.subplots(1, 2, figsize=(20,10))
sns.barplot(x='Scores', y='Features', data=gbc_df, palette='Blues_r', ax=ax[0])
ax[0].set_title('Significant Predictors from Gradient Boosting (With Child)', fontsize=14)
sns.barplot(x='Scores', y='Features', data=noch_gbc_df, palette='Blues_r', ax=ax[1])
ax[1].set_title('Significant Predictors from Gradient Boosting (Without Child)', fontsize=14)
fig.savefig('/Users/Ming/Documents/GitHub/wellcome_dissertation/WellcomePy/py_image/comb_fimpt.png')
plt.close()

### Combine Comparison Plot

In [30]:
fig, ax = plt.subplots(1,2,figsize=(20,10))
sns.lineplot(x='Model', y='Accuracy', data=comp_mod_df_ch, sort=False, ax=ax[0])
ax[0].set_title('Comparison of DT, RF, and GB accuracy (with child)', fontsize=14)
sns.lineplot(x='Model', y='Accuracy', data=comp_mod_df_noch, sort=False, ax=ax[1])
ax[1].set_title('Comparison of DT, RF, and GB accuracy (without child)', fontsize=14)
fig.savefig('/Users/Ming/Documents/GitHub/wellcome_dissertation/WellcomePy/py_image/mod_comp.png')
plt.close()

In [32]:
comp_mod_df_ch

,Model,Accuracy
0,DT,0.81156
1,RF,0.89133
2,GB,0.89041


In [33]:
comp_mod_df_noch

,Model,Accuracy
0,DT,0.74838
1,RF,0.84619
2,GB,0.84619


### CV Score

In [222]:
dtcv_ch = tree_cv(ch_train_feature, ch_train_label, dtc, 'Decision Tree')

[0.80744758 0.80383225 0.80759219 0.8054953  0.81083231]


=== Mean Accuracy Score ===
Mean Accuracy Score - Decision Tree: 0.81


In [223]:
rfcv_ch = tree_cv(ch_train_feature, ch_train_label, rfc, 'Random Forest')

[0.88778019 0.88843095 0.88886479 0.88850325 0.889146  ]


=== Mean Accuracy Score ===
Mean Accuracy Score - Random Forest: 0.89


In [224]:
gbcv_ch = tree_cv(ch_train_feature, ch_train_label, gbc, 'Random Forest')

[0.88893709 0.88879248 0.88958785 0.88864787 0.89066455]


=== Mean Accuracy Score ===
Mean Accuracy Score - Random Forest: 0.89


### TRUSCI_IDX Investigation

In [79]:
df = open_csv('full_data.csv')

In [80]:
df['CTRY_NAME'] = df['CTRY'].replace(country)

In [ ]:
df['RGN_NAME'] = df['RGN'].replace(rgn)

In [199]:
def trusci_idx_ctry(df, ctry_1, ctry_2, ctry_3, ctry_4, ctry_5):
    country = df[(df['CTRY_NAME'] == ctry_1) | (df['CTRY_NAME'] == ctry_2) | (df['CTRY_NAME'] == ctry_3) | (df['CTRY_NAME'] == ctry_4) | (df['CTRY_NAME'] == ctry_5)]
    grouped = country.groupby(['CTRY_NAME'])['TRUSCI_IDX'].quantile(.75).reset_index().sort_values(by='TRUSCI_IDX', ascending=False).iloc[:,0].tolist()
    return (country, grouped)

In [200]:
west_eu, west_eu_idx = trusci_idx_ctry(df, 'France', 'Germany', 'Switzerland', 'Netherlands', 'Belgium')
east_eu, east_eu_idx = trusci_idx_ctry(df, 'Russia', 'Belarus', 'Ukraine', 'Poland', 'Romania')
east_as, east_as_idx = trusci_idx_ctry(df, 'China', 'Japan', 'South Korea', 'Mongolia', 'Taiwan')
south_am, south_am_idx = trusci_idx_ctry(df, 'Brazil', 'Peru', 'Venezuela', 'Argentina', 'Bolivia')
sea, sea_idx = trusci_idx_ctry(df, 'Indonesia', 'Thailand', 'Malaysia', 'Philippines', 'Vietnam')
north_af, north_ar_idx = trusci_idx_ctry(df, 'Egypt', 'Morocco', 'Algeria', 'Libya', 'Tunisia')

In [204]:
fig, ax = plt.subplots(nrows=2,ncols=3,figsize=(16,10))
sns.boxplot(x='CTRY_NAME', y='TRUSCI_IDX', data=west_eu, ax=ax[0,0], order=west_eu_idx)
ax[0,0].set_title('TRUSCI_IDX in Western Europe Countries')
sns.boxplot(x='CTRY_NAME', y='TRUSCI_IDX', data=east_eu, ax=ax[0,1], order=east_eu_idx)
ax[0,1].set_title('TRUSCI_IDX in Eastern Europe Countries')
sns.boxplot(x='CTRY_NAME', y='TRUSCI_IDX', data=east_as, ax=ax[0,2], order=east_as_idx)
ax[0,2].set_title('TRUSCI_IDX in East Asia Countries')
sns.boxplot(x='CTRY_NAME', y='TRUSCI_IDX', data=south_am, ax=ax[1,0], order=south_am_idx)
ax[1,0].set_title('TRUSCI_IDX in South America Countries')
sns.boxplot(x='CTRY_NAME', y='TRUSCI_IDX', data=sea, ax=ax[1,1], order=sea_idx)
ax[1,1].set_title('TRUSCI_IDX in Southeast Asia Countries')
sns.boxplot(x='CTRY_NAME', y='TRUSCI_IDX', data=north_af, ax=ax[1,2], order=north_ar_idx)
ax[1,2].set_title('TRUSCI_IDX in North Africa Countries')
fig.savefig('/Users/Ming/Documents/GitHub/wellcome_dissertation/WellcomePy/py_image/trusci_idx_ctry.png')
plt.close()